In [2]:
import requests
import os

url = "https://buildingtransparency.org/api/rest-auth/login"

querystring = {"tokenless":"false"}

payload = {
    "username": os.getenv("OPENEPD_USER"),
    "password": os.getenv("OPENEPD_PW")
}
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

response = requests.post(url, json=payload, headers=headers, params=querystring)

response_data = response.json()

#print(response_data)

auth_key = response.json()['key']

print(auth_key)

KeyError: 'key'

In [2]:
headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+auth_key
}

#ndustry_epd_url = "https://buildingtransparency.org/api/industry_epds"
product_epd_url = "https://buildingtransparency.org/api/epds"

querystring = {"page_size":"250"}

#indus_response = requests.get(industry_epd_url, headers=headers, params=querystring)
prod_response = requests.get(product_epd_url, headers=headers, params=querystring)

#indus_results = indus_response.json()
prod_results = prod_response.json()

In [3]:
len(prod_results)

250

In [5]:
#total_pages = int(indus_response.headers['X-Total-Pages'])
total_pages2 = int(prod_response.headers['X-Total-Pages'])

#print('Industry EPD Pages: ', total_pages)
print('Product EPD Pages: ', total_pages2)

Product EPD Pages:  722


In [21]:
product_epds = []

for i in range(0,2):
        querystring = {
                "page_number": i+1,
                "page_size":"250"}
        response = requests.get(product_epd_url, headers=headers, params=querystring)
        epd_data = response.json()
        for epd in epd_data:
                product_epds.append(epd)
        print("successfully completed page "+str(i+1))

successfully completed page 1
successfully completed page 2


In [26]:
def get_page(page):    
    epd_lst = []

    querystring = {
            "page_number": page,
            "page_size":"250"}
    response = requests.get(product_epd_url, headers=headers, params=querystring)
    epd_data = response.json()
    for epd in epd_data:
            epd_lst.append(epd)
    print("successfully loaded page "+str(page))

    return epd_lst

In [ ]:
product_epds[250]

In [39]:
from datetime import datetime
import requests
#category = ['GWP-total','GWP-biogenic','GWP-fossil','GWP-IOBC']

headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+auth_key
}

product_epd_url = "https://buildingtransparency.org/api/epds"
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"

EPDs = []
EC3_epds = []
product_epds = []
none = "Unmapped"
page = 0

for x in range(0,10):
    if x % 250 == 0:
        page =+ 1
        EC3_epds.extend(get_page(page))


    date_validity_ends_str = EC3_epds[x]['date_validity_ends']
    date_validity_ends = datetime.strptime(date_validity_ends_str, '%Y-%m-%d')

    if date_validity_ends < datetime.now():
        print(f"EPD {x} expired, skipping...")
        continue

    xpdID = EC3_epds[x]['open_xpd_uuid']
    response_impact = requests.get(impact_info_url+xpdID, headers=headers)
    impact = response_impact.json()


    ############################### DATA PATHS #################################
    EPDID = EC3_epds[x]['category']['id']
    EPDname = EC3_epds[x]['category']['display_name']
    desc = EC3_epds[x]['category']['description']
    seLi = EC3_epds[x]['reference_service_life']
    pcr_name = EC3_epds[x]['pcr']['name']
    pcr_id = EC3_epds[x]['pcr']['id']
    short_link = EC3_epds[x]['pcr']['short_link']
    created_on = EC3_epds[x]['category']['created_on']

    thckqty = EC3_epds[x]['thickness']
    thckU = EC3_epds[x]['thickness_per_declared_unit']

    tMode = EC3_epds[x]['category']['default_transport_mode']
    dist = EC3_epds[x]['category']['default_distance']

    dens = EC3_epds[x]['density']
    sdens = EC3_epds[x]['specific_density']

    unit = EC3_epds[x]['category']['declared_unit']
    lcia = list(EC3_epds[x]['pcr']['lcia_requirements'].keys())[0]
    



    material_data = {
    "material_id": EPDID,
    "validity": {
      "gte": created_on,
      "lte": date_validity_ends_str
    },
    "display_name": EPDname,
    "source": "Product Specific EPDs",
    "declared_unit": unit,
    "A4_transportation": {
      "mode": tMode,
      "distance": {
        "qty": dist,
        "unit": "km"
      }
    },
    "service_life": seLi,
    "waste_rate": none,
    "density": {
      "qty": dens,
      "unit": "kg/m3"
    },
    "item_mass": {
      "qty": 1.0,
      "unit": "kg"
    },
    "impacts": {
      "EN15804": {
        "gwp_total": {
          "A1_A3": 2.1,
          "C2": 0.1,
          "C3": 0.05,
          "C4": 0.01,
          "D1": -0.5
        },
        "gwp_fossil": {
          "A1_A3": 2.0,
          "C2": 0.1,
          "C3": 0.04,
          "C4": 0.01,
          "D1": -0.48
        }
      }
    },
    "characteristics": {}
  }
    
    EPDs.append(material_data)
    print(f'json #{x} is done')



successfully loaded page 1
json #0 is done
json #1 is done
json #2 is done
json #3 is done
json #4 is done
json #5 is done
json #6 is done
json #7 is done
json #8 is done
json #9 is done


In [41]:
EPDs[0]

{'material_id': 'to be assigned later',
 'display_name': 'Ready Mix',
 'source': 'Unmapped',
 'description': 'Concretes to be mixed and then poured on-site,',
 'reference_pcr': {'name': 'North American PCR for Concrete',
  'id': '64e661dd8d894db3a1bb327f08edae1f',
  'link': 'cqd.io/p/ec37zjz1ws'},
 'thickness': {'qty': None, 'unit': None},
 'service_life': None,
 'waste_rate': 'Unmapped',
 'transportation': {'mode': 'concrete mixer truck', 'distance': '51.49888 km'},
 'density': {'qty': '2400 kg / m^3', 'unit': 'Unmapped'},
 'linear_density': {'qty': None, 'unit': 'Unmapped'},
 'declared_unit': '1 m3',
 'lcia_method': 'TRACI 2.1',
 'background_database': 'Unmapped',
 'impacts': {'gwp_total': {},
  'gwp_biogenic': {},
  'gwp_fossil': {},
  'gwp_iobc': {}},
 'characteristics': {}}

In [12]:
#epd_data[0]['open_xpd_uuid']
len(product_epd_ids)
product_epd_ids[0]



'ec34u6kx'

In [13]:
import requests

impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"+"ec36urza"

response_impact = requests.get(impact_info_url, headers=headers)
response_impact.json()


{'id': 'ec36urza',
 'ref': 'https://openepd.buildingtransparency.org/api/epds/EC36URZA',
 'ec3': {'uaGWP_a1a2a3_traci21': 437.8297093653758,
  'uaGWP_a1a2a3_ar5': 439.1545751006035,
  'category': 'Concrete >> Shotcrete',
  'manufacturer_specific': True,
  'plant_specific': True,
  'product_specific': True,
  'batch_specific': False,
  'supply_chain_specificity': 0.0,
  'original_data_format': 'pdf'},
 'openepd_version': '0.1',
 'doctype': 'OpenEPD',
 'version': None,
 'language': 'en',
 'private': False,
 'declaration_url': 'https://cqd.io/e/ec36urzas7',
 'lca_discussion': '\n\n## Product Component Notes\ncrushed aggregate (ASTM C33), natural aggregate (ASTM C33), Portland cement (ASTM C150), slag cement (ASTM C989), fly ash (ASTM C618), admixture (ASTM C494), batch water (ASTM C1602)\n\n## LCA Notes\nTransportation: Transportation of these materials from supplier to the gate\nof the concrete producer.\nManufacturing (core processes): Energy used to store, batch, mix, and\ndistribute t

In [ ]:
    material_data = {
        "material_id": 'to be assigned later',
        "display_name": EPDname,
        "source": none,
        "description": f'{desc},',
        #"validity": date_validity_ends_str,
        "reference_pcr": {
            "name": pcr_name,
            "id": pcr_id,
            "link": short_link
        },
        "thickness": {
            "qty": thckqty,
            "unit": thckU
        },
        "service_life": seLi,
        "waste_rate": none,
        "transportation": {
            "mode": tMode,
            "distance": dist
        },
        "density": {
            "qty": dens,
            "unit": none
        },
        "linear_density": {
            "qty": sdens,
            "unit": none
        },
        "declared_unit": unit,
        "lcia_method": lcia,
        "background_database": none,
        "impacts": {
            "gwp_total": {
                #"A1": A11, "A2": A12, "A3": A13, "A4": A14, "A5": A15, "C1": C11,
                #"C2": C12, "C3": C13, "C4": C14, "D": D1, "D1": D11, "D2": D12
            },
            "gwp_biogenic": {
                #"A1": A21, "A2": A22, "A3": A23, "A4": A24, "A5": A25, "C1": C21,
                #"C2": C22, "C3": C23, "C4": C24, "D": D2, "D1": D21, "D2": D22
            },
            "gwp_fossil": {
                #"A1": A31, "A2": A32, "A3": A33, "A4": A34, "A5": A35, "C1": C31,
                #"C2": C32, "C3": C33, "C4": C34, "D": D3, "D1": D31, "D2": D32
            },
            "gwp_iobc": {
                #"A1": A41, "A2": A42, "A3": A43, "A4": A44, "A5": A45, "C1": C41,
                #"C2": C42, "C3": C43, "C4": C44, "D": D4, "D1": D41, "D2": D42
            }
        },
        "characteristics": {}
    },